### Tools and Routing with Langchain
Langchain allows you to build tools in into agents and route to which tool will be used

In [1]:
from langchain.agents.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI

In [2]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [4]:
search.name, search.description, search.args

('search',
 'search(query: str) -> str - Search for weather online',
 {'query': {'title': 'Query', 'type': 'string'}})

In [7]:
from pydantic.v1 import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [8]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [9]:
search.run("Sf")

'42f'

In [10]:
search

StructuredTool(name='search', description='search(query: str) -> str - Search for the weather online.', args_schema=<class '__main__.SearchInput'>, func=<function search at 0x16a2a19e0>)

In [12]:
import requests
from pydantic.v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [13]:
get_current_temperature.name, get_current_temperature.description, get_current_temperature.args

('get_current_temperature',
 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 {'latitude': {'title': 'Latitude',
   'description': 'Latitude of the location to fetch weather data for',
   'type': 'number'},
  'longitude': {'title': 'Longitude',
   'description': 'Longitude of the location to fetch weather data for',
   'type': 'number'}})

In [14]:
from langchain.tools.render import format_tool_to_openai_function

In [15]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'title': 'OpenMeteoInput',
  'type': 'object',
  'properties': {'latitude': {'title': 'Latitude',
    'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'title': 'Longitude',
    'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [16]:
get_current_temperature({"latitude": 13, "longitude": 14})

'The current temperature is 21.5°C'

In [17]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [18]:
search_wikipedia.name

'search_wikipedia'

In [19]:
search_wikipedia({"query": "langchain"})

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n\n\nPage: Sentence embedding\nSummary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers whi

In [20]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [21]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [22]:
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


AttributeError: 'super' object has no attribute 'parse_obj'

#### Routing in OpenAI tools
Routing allows us to select which tools to use when performing an operation and calls the appropriate function

In [24]:
functions = [
    format_tool_to_openai_function(t) for t in [search_wikipedia, get_current_temperature]
]

In [25]:
functions

[{'name': 'search_wikipedia',
  'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
  'parameters': {'title': 'search_wikipediaSchemaSchema',
   'type': 'object',
   'properties': {'query': {'title': 'Query', 'type': 'string'}},
   'required': ['query']}},
 {'name': 'get_current_temperature',
  'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
  'parameters': {'title': 'OpenMeteoInput',
   'type': 'object',
   'properties': {'latitude': {'title': 'Latitude',
     'description': 'Latitude of the location to fetch weather data for',
     'type': 'number'},
    'longitude': {'title': 'Longitude',
     'description': 'Longitude of the location to fetch weather data for',
     'type': 'number'}},
   'required': ['latitude', 'longitude']}}]

In [35]:
model = ChatOpenAI(model="mistralai/Mistral-7B-Instruct-v0.2")

In [36]:

from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model


In [37]:
from langchain.agents.output_parsers.openai_functions import OpenAIFunctionsAgentOutputParser

In [38]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [39]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [40]:
result

AgentFinish(return_values={'output': "Oh, you're asking about the weather in San Francisco, are you? Let me check my calendar... Oh, right, I don't have one. But I do have a magical ability to know the weather anytime, anywhere. So, let me consult my meteorological crystal ball (patent pending):\n\nIt's currently a brisk 58 degrees Fahrenheit (14 degrees Celsius) with a light breeze coming in from the Pacific. And no, I don't need to look it up. I'm that good. But if you really want to make my day, bring me a latte and a croissant. Preferably one with chocolate. Just saying."}, log="Oh, you're asking about the weather in San Francisco, are you? Let me check my calendar... Oh, right, I don't have one. But I do have a magical ability to know the weather anytime, anywhere. So, let me consult my meteorological crystal ball (patent pending):\n\nIt's currently a brisk 58 degrees Fahrenheit (14 degrees Celsius) with a light breeze coming in from the Pacific. And no, I don't need to look it up

In [ ]:
get_current_temperature(result.tool_input)

In [31]:
from langchain.schema.agent import AgentFinish # what is entailed within agent finish?
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [34]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

NotFoundError: Error code: 404 - {'model': 'mistralai/Mistal-7B-Instruct-v0.2', 'job_id': '', 'request_id': '84e4b6531fa5634d-LHR', 'error': 'mistralai/Mistal-7B-Instruct-v0.2 is not a supported model. Please visit https://api.together.xyz to see a complete list of supported models.'}